# Collaborative filtering

- User-based CF

In [1]:
import dataset
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import mean_squared_error

data = dataset.Dataset()
train = data.train
test = data.test
item = data.item

In [2]:
#Creates cosine similarity matrices for users and movies. 
user_similarity = cosine_similarity(train)
user_similarity.shape

(943, 943)

In [4]:
## 사람마다 평균 높게 주는 사람 각기 다르므로, bias 제거해주기 위함
mean_user_rating = train.mean(axis=1)[:, np.newaxis] 
ratings_diff = train - mean_user_rating
user_pred = mean_user_rating + user_similarity.dot(ratings_diff) / np.array([user_similarity.sum(axis=1)]).T

In [5]:
from math import sqrt
#Root Mean Squared Error for validation.
def rmse(pred, test):
    pred = pred[test.nonzero()].flatten() 
    test = test[test.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, test))

print("Collaborative Filtering RMSE")
print('User-based: ', rmse(user_pred, train)) 

Collaborative Filtering RMSE
User-based:  2.720442982119774


# k_recommend

In [3]:
def user_k_recommend(user_id,k):
    list=[]
    for index, j in enumerate(user_pred[user_id-1]):
        list.append((index+1,j))
    movie_sim = pd.DataFrame(list, columns=['movie id','similarity'])
    result_pd = pd.merge(movie_sim,item.iloc[:,:2], on='movie id')
    return result_pd.sort_values('similarity', ascending=False)[:k]['movie title']

In [6]:
user_k_recommend(3,5)

257              Contact (1997)
49             Star Wars (1977)
299        Air Force One (1997)
301    L.A. Confidential (1997)
287               Scream (1996)
Name: movie title, dtype: object